In [1]:
import os

from dotenv import load_dotenv
_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import AzureChatOpenAI
import httpx

In [3]:
from fastmcp import Client

mcp_client = Client("02_agent_mcp_server.py")
await mcp_client.__aenter__()

mcp_tools = await mcp_client.list_tools()

tools = [
    {
        "name": tool.name,
        "description": tool.description,
        "parameters": getattr(tool, 'inputSchema', {"type": "object", "properties": {}})
    }
    for tool in mcp_tools
]
tools

[{'name': 'get_weather',
  'description': 'Get the current weather in a given location\n\nArgs:\n    latitude: Latitude of the location for which to get the weather\n    longitude: Longitude of the location for which to get the weather\n\nReturns:\n    JSON with current weather information',
  'parameters': {'properties': {'latitude': {'type': 'string'},
    'longitude': {'type': 'string'}},
   'required': ['latitude', 'longitude'],
   'type': 'object'}},
 {'name': 'facts_from_wikipedia',
  'description': 'Get Wikipedia facts about a given topic, location, person etc.\n\nArgs:\n    topic: A topic, location, person etc. to search for on Wikipedia\n\nReturns:\n    JSON with Wikipedia information',
  'parameters': {'properties': {'topic': {'type': 'string'}},
   'required': ['topic'],
   'type': 'object'}}]

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t['name']: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        print(result)
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    async def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:
                result = "bad tool call"
            else:
                r = await mcp_client.call_tool(t['name'], t['args'])
                if hasattr(r, 'content') and r.content:
                    result = r.content[0].text
                else:
                    result = str(r)
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = AzureChatOpenAI(
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
            azure_endpoint=os.environ["AZURE_ENDPOINT"],
            api_version="2024-08-01-preview",
            deployment_name=os.environ['AZURE_MODEL_NAME'],
            temperature=0,
            model=os.environ['AZURE_MODEL_NAME'],
            http_client=httpx.Client(verify=False),
            http_async_client=httpx.AsyncClient(verify=False),
        )
abot = Agent(model, tools, system=prompt)

In [7]:
messages = [HumanMessage(content="What is the weather in warsaw?")]
result = await abot.graph.ainvoke({"messages": messages})

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 217, 'total_tokens': 242, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-CToW5GrOIff4varyeYbPXrdT3oClb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}} id='lc_run--bb284f87-163e-4a73-9b2c-a5036f8952d9-0' tool_calls=[{'name': 'get_weather', 'args': {'latitude': '52.2297', 'longitude':

In [8]:
result['messages'][-1].content

{'messages': [HumanMessage(content='What is the weather in warsaw?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 217, 'total_tokens': 242, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-CToW5GrOIff4varyeYbPXrdT3oClb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id=